# UC Case Synthesis

This notebook synthetises the case of a Unit Commitment problem.

The source data comes from Grid Optimization Competition (GO Competition) and is available at https://gocompetition.energy.gov/challenges/challenge-2

Here we use the dataset: C2FEN31777, Scenario 5, 31777 Buses, X Lines

In [1]:
import numpy as np

import ams

import json

In [2]:
ams.config_logger(stream_level=20)

There are five scenarios in the dataset, and the case is regarded as one area with 5 hourly intervals.

The Load factors are calculated by summing the power values for each case, storing these sums, and then normalizing them by dividing each sum by the maximum sum across all cases. This results in relative loading factors for comparison.

In [3]:
cases = [
    './C2FEN31777/scenario_011/case.raw',
    './C2FEN31777/scenario_012/case.raw',
    './C2FEN31777/scenario_015/case.raw',
    './C2FEN31777/scenario_019/case.raw',
    './C2FEN31777/scenario_051/case.raw',
]

sd_data = np.zeros(len(cases), dtype=float)

for i, case in enumerate(cases):
    sp = ams.load(case, setup=True, default_config=True, no_output=True)
    psum = sp.PQ.p0.v.sum()
    sd_data[i] = psum

sd_data[:] = sd_data / sd_data.max()
sd_data

Parsing input file "./C2FEN31777/scenario_011/case.raw"...
Input file parsed in 7.3241 seconds.
System set up in 0.0861 seconds.
Parsing input file "./C2FEN31777/scenario_012/case.raw"...
Input file parsed in 7.3992 seconds.
System set up in 0.0866 seconds.
Parsing input file "./C2FEN31777/scenario_015/case.raw"...
Input file parsed in 7.3907 seconds.
System set up in 0.0882 seconds.
Parsing input file "./C2FEN31777/scenario_019/case.raw"...
Input file parsed in 7.3758 seconds.
System set up in 0.0853 seconds.
Parsing input file "./C2FEN31777/scenario_051/case.raw"...
Input file parsed in 7.3412 seconds.
System set up in 0.1406 seconds.


array([1.        , 0.85034375, 0.94469362, 0.86164711, 0.96305627])

## Supplerary Data Description

### System Parameters

| Field      | Unit | Description                                                                 |
|------------|------|-----------------------------------------------------------------------------|
| delta      | h    | duration of the base case                                                   |
| deltactg   | h    | duration of each contingency case                                           |
| deltar     | h    | duration (for ramping) from the given operating point prior to the base case to base case operating point |
| duration   | h    | duration (for ramping) from observation of a contingency to post-contingency operating point |

### Generators

| Field      | Unit         | Description                                                                 |
|------------|--------------|-----------------------------------------------------------------------------|
| bus        | -            | bus number                                                                  |
| id         | -            | generator identifier                                                        |
| suqual     | -            | allowed to start up in base case                                            |
| sdqual     | -            | allowed to shut down in base case                                           |
| suqualct   | -            | allowed to start up in contingencies                                        |
| sdqualct   | -            | allowed to shut down in contingencies                                       |
| prumax     | pu/h         | maximum ramp up rate in base case                                           |
| prdmax     | pu/h         | maximum ramp down rate in base case                                         |
| prumaxctg  | pu/h         | maximum ramp up rate in contingencies                                       |
| prdmaxctg  | pu/h         | maximum ramp down rate in contingencies                                     |
| oncost     | USD/h        | fixed operating cost                                                        |
| sucost     | USD          | startup cost                                                                |
| sdcost     | USD          | shutdown cost                                                               |
| cblocks    | USD/pu-h     | a convex piecewise linear cost function, where pmax and c are the maximum power output and cost of the block, respectively |

In [4]:
file_name = './C2FEN31777/scenario_011/case.json'
# Open the JSON file
with open(file_name, 'r') as file:
    # Load the contents of the file into a Python dictionary
    data = json.load(file)

data.keys()

dict_keys(['systemparameters', 'loads', 'generators', 'lines', 'transformers', 'pcblocks', 'qcblocks', 'scblocks'])

In [5]:
# use the most heavy load one as base case
sp = ams.load(cases[sd_data.argmax()],
              setup=False, default_config=True, no_output=True)

Parsing input file "./C2FEN31777/scenario_011/case.raw"...
Input file parsed in 7.5264 seconds.


In [6]:
# --- Generator ---
for gen in data['generators']:
    # get generator idx
    stg_idx = sp.StaticGen.find_idx(keys=['bus', 'subidx'],
                                    values=[[gen['bus']], [int(gen['id'])]])[0]

    # alter ramping data
    sp.StaticGen.alter(src='R30', idx=stg_idx, value=gen['prumax']/2)

    # add cost data
    cblock = gen['cblocks']
    c1 = np.mean([cblock[i]['c'] for i in range(len(cblock))])
    sp.add(model='GCost', param_dict=dict(gen=stg_idx,
                                          csu=gen['sucost'], csd=gen['sdcost'],
                                          c0=gen['oncost'], c1=c1, c2=0))

    # gen enforce on/off status
    if gen['suqual'] == 0:
        sp.StaticGen.alter(src='uf', idx=stg_idx, value=-1)
    if gen['sdqual'] == 0:
        sp.StaticGen.alter(src='df', idx=stg_idx, value=1)

# --- Load ---
# NOTE: here we know the data['loads'] and ss.PQ are one-to-one mapping
for load, pq_idx in zip(data['loads'], sp.PQ.idx.v):
    cblock = load['cblocks']
    cdp = cblock[0]['c']  # pick the first point as cdp
    sp.add(model='DCost', param_dict=dict(pq=pq_idx, cdp=cdp))

# --- Region ---
# NOTE: add two regions, where the first one is the actually used one
sp.add(model='Region', param_dict=dict(idx='Zone_1'))
sp.add(model='Region', param_dict=dict(idx='Zone_2'))
sp.Bus.zone.v = sp.Bus.n * ['Zone_1']  # overwrite Bus.zone

# --- UC load factor ---
for sd in sd_data:
    sp.add(model='UCTSlot', param_dict=dict(sd=f'{sd:.2f} ,0'))

# --- ED load factor ---
# NOTE: for placeholder
ug = ', '.join(map(str, [1] * sp.StaticGen.n))
for sd in sd_data:
    sp.add(model='EDTSlot', param_dict=dict(sd=f'{sd:.2f} ,0', ug=ug))

# --- Non-spinning reserve ---
# NOTE: use 0.1 for all regions
for z in sp.Region.idx.v:
    sp.add(model='NSR', param_dict=dict(demand=0.1, zone=z))

# --- Non-spinning reserve cost ---
# NOTE: use 0.1 for all generators
for stg_idx in sp.PV.idx.v + sp.Slack.idx.v:
    sp.add(model='NSRCost', param_dict=dict(gen=stg_idx, cnsr=0.1))

# --- Spinning reserve ---
# NOTE: use 0.3 for all regions
for z in sp.Region.idx.v:
    sp.add(model='SR', param_dict=dict(demand=0.3, zone=z))

# --- Spinning reserve cost ---
# NOTE: use 0.1 for all generators
for stg_idx in sp.PV.idx.v + sp.Slack.idx.v:
    sp.add(model='SRCost', param_dict=dict(gen=stg_idx, csr=0.1))

# --- Secondary Frequency Regulation Reserve ---
# NOTE: use 0.02 for both du and dd for all regions
for z in sp.Region.idx.v:
    sp.add(model='SFR', param_dict=dict(du=0.02, dd=0.02, zone=z))

# --- Secondary Frequency Regulation Reserve cost ---
# NOTE: use 0 for both cru and crd for all generators
for stg_idx in sp.PV.idx.v + sp.Slack.idx.v:
    sp.add(model='SFRCost', param_dict=dict(gen=stg_idx, cru=0, crd=0))

In [7]:
sp.setup()

System set up in 0.1324 seconds.


True

In [8]:
sp.UC.init()

<UC> initialized in 37.9986 seconds.


True

In [9]:
sp.UC.run(solver='GUROBI', ignore_dpp=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-25


<UC> solved as optimal in 44.9931 seconds, converged in 10298 iterations with GUROBI.


True

In [10]:
for i, edt in enumerate(sp.EDTSlot.idx.v):
    sp.EDTSlot.alter(src='ug', idx=edt, value=sp.UC.ugd.v[:, i])

In [11]:
ams.io.xlsx.write(sp, './goc31777_uced.xlsx', overwrite=True)

xlsx file written to "./goc31777_uced.xlsx"


True